# ⭐ Pomysł 3: pierwszy akapit wikipedii

## The goal

- wpleść elementy o chorwacji
- założenie: duże walki o pierwszy akapit, sugeruje bardzo dużo
- jak pierwszy akapit się różni między "dobrymi" a "złymi"?
- wymień ważne różnice faktograficzne między pierwszym akapitem a skompresowaną róznicę. Czy jest jakaś zależność między osią / aliantami?

## The links

Colab: https://colab.research.google.com/drive/1luke25I1eo9-pIylnd02-aulI3TU3l5H?authuser=1&pli=1#scrollTo=Qa1cwQWVHnih

## The procedure

1. 100 postaci historycznych najbardziej znane (WWII)
2. weź NAJTAŃSZY MODEL (GPT 3.5) ewentualnie polskiego bielika, LLamę
3. bierzesz pierwszy akapit, każesz faktograficznie skompresować resztę (max 300 słów), "podsumuj biogram bazując wyłącznie na tekście"
4. przesyłasz pierwszy akapit i skompresowany artykuł, pytasz: czym się różnią, które są zawarte, a które pominięte? Pozytywne / negatywne? w czterech punktach (do 300-słów) wymień co zawarte, co pominięte, "BAZUJĄC NA TEKSCIE"
5. embeddingi tych 4 punktów dla każdej z postaci
6. klastrujesz
7. czy w tej różnicy doboru faktów historiograficznych dla postaci da się wyczuć z której strony pochodzą?
   prosta klasyfikacja - ludzie osi i aliantów

na początek: churchill, roosvelt, stalin, hitler, hirohito

# The data

## getting wikipedia entries

In [1]:
from urllib.request import urlopen
from urllib.parse import quote

import json
from bs4 import BeautifulSoup


In [2]:
# Wikipedia API on getting the contents: 4 ways: https://www.mediawiki.org/wiki/API:Get_the_contents_of_a_page
# the two first return wikitext which seems to be very sophisitcated WikiMedia markup language, universally difficult to parse
# 3rd method returns html

core_url = 'https://en.wikipedia.org/w/api.php'
def get_request_parameters_stirng(page: str):
  # name =  page.encode('utf-8')
  return f'?action=parse&page={quote(page)}&prop=text&formatversion=2&format=json'


def get_page_content(page):

  print(page)
  url = f'{core_url}{get_request_parameters_stirng(page)}'
  response = urlopen(url)
  json_content = json.load(response)
  html_content = json_content["parse"]["text"]
  return html_content


# Preprocessing

In [3]:
def get_paragraphs(html_string:str):
  # preliminary cleaning
  cleared_html_string = html_string.replace("\n", "")
  soup = BeautifulSoup(cleared_html_string, 'html.parser')
  all_paragraphs = [paragraph.get_text() for paragraph in soup.find_all("p") if paragraph.get_text().strip() and len(paragraph.get_text().strip().split(" ")) > 1]
  return all_paragraphs

In [4]:

def get_data(page_name):
  # fetch page
  print(f'Getting page: {page_name}')
  html_string = get_page_content(page_name)

  # preprocess page
  paragraphs = get_paragraphs(html_string=html_string)
  print(f"{len(paragraphs)} paragraphs found")

  return paragraphs


In [5]:
#test 

html_content = get_data('Charles_de_Gaulle')
html_content

Getting page: Charles_de_Gaulle
Charles_de_Gaulle
157 paragraphs found


['Charles André Joseph Marie de Gaulle[a][b] (22 November 1890\xa0– 9 November 1970) was a French military officer and statesman who led the Free French Forces against Nazi Germany in World War II and chaired the Provisional Government of the French Republic from 1944 to 1946 to restore democracy in France. In 1958, amid the Algerian War, he came out of retirement when appointed Prime Minister by President René Coty. He rewrote the Constitution of France and founded the Fifth Republic after approval by referendum. He was elected President of France later that year, a position he held until his resignation in 1969.',
 "Born in Lille, he was a decorated officer of the First World War, wounded several times and taken prisoner by the Germans. During the interwar period, he advocated mobile armoured divisions. During the German invasion of May 1940, he led an armoured division that counterattacked the invaders; he was then appointed Undersecretary for War. Refusing to accept his government'

## LLM setup

In [6]:
from ollama import generate, GenerateResponse

def get_response(prompt):
  """Helper, use ollama API to return a generated response"""
  response: GenerateResponse = generate(model='llama3.2:3b', prompt=prompt, options={"temperature": 1}).response
  return response

### Ollama on Google Colab

In [7]:
# !pip install colab-xterm
# %load_ext colabxterm

In [8]:
# %xterm
## in it, go
# 👉🏼 curl -fsSL https://ollama.com/install.sh | sh && ollama serve &
# 👉🏼 ollama pull llama3.2:3b                      # (or any other) model for summarization
# 👉🏼 ollama pull jina/jina-embeddings-v2-base-en  # (or any other) model for embeddings


In [9]:
# %xterm


In [10]:
# 👉🏼 ollama pull llama3.2:3b                      # (or any other) model for summarization
# 👉🏼 ollama pull jina/jina-embeddings-v2-base-en  # (or any other) model for embeddings
# !ollama pull llama3.2:3b
# !ollama pull jina/jina-embeddings-v2-base-en
# !pip install ollama

In [11]:
!ollama list

NAME                                      ID              SIZE      MODIFIED     
jina/jina-embeddings-v2-base-en:latest    377193292084    274 MB    4 weeks ago     
llama3.2:3b                               a80c4f17acd5    2.0 GB    2 months ago    


In [12]:
import os
# os.environ['OLLAMA_HOST'] = '127.0.0.1:11434'

## factual compression of a biogram

In [13]:
def compress(paragraphs: str):

  content = """


""".join(paragraphs)
  prompt = f'''Please, compress this biography factually.

Use only facts from the texts provided, and convey them in neutral tone, faithful to neutral point of view.
Replace direct references to the biography's subject with a word 'person'
DO NOT include a person's name in the answer (inc. their name, surname, acronym - anything that identifies them
DO NOT include any other content besides the compressed content, no intro or outro to your answer.
DO NOT include follow-up questions.

{content}'''
  return get_response(prompt)

## comparisons

In [14]:
def get_comparison(text_a, text_b):
  prompt = f"""Compare the summarizing paragraph (text A) with facts from the biography (text B).
  Which parts of text B does text A emphazie?

  Is the summary balanced? what kind of omissions are made?

Provide your answer in up to 4 bulletpoints
Each of the points should include mention of which text (text A or text B) is being considered
The answer should be consist of up to 300 characters total.

- DO NOT include a person's name, surname or any direct reference to them
- DO NOT mention the exact details
- DO NOT include anything else than the points made.

text 1:
{text_a}

text 2:
{text_b}
"""
  answer = get_response(prompt)
  return answer


## embeddings

In [15]:
import ollama

def get_embeddings(text):
  return ollama.embeddings(model='jina/jina-embeddings-v2-base-en', prompt=text).embedding

## pipeline

In [16]:
from dataclasses import dataclass
from typing import Optional

@dataclass
class PageRecord:
  name: str
  paragraphs: list
  abstract: str
  compressed_bio: str
  comparison: str
  comparison_embedding: Optional[str]

  def recompress(self):
    self.compressed_bio = compress(self.paragraphs[1:])
    return self

  def recompare(self):
    self.comparison = get_comparison(self.abstract, self.compressed_bio)
    return self

def process_data(page_name, paragraphs):
  # compress the remaining bio
  [first, the_rest] = paragraphs[0], paragraphs[1:]
  print(f"Compressing the article...")
  compressed_bio = compress(the_rest)

  # compare abstract and the compressed_bio
  print(f"Generating comparison...")
  comparison = get_comparison(first, compressed_bio)

  print(f"Generating embeddings for comparison...")
  comparison_embedding = get_embeddings(comparison)
  return PageRecord(name=page_name, paragraphs=paragraphs, abstract=first, compressed_bio=compressed_bio, comparison=comparison, comparison_embedding=comparison_embedding)


## call

In [17]:

articles = [
    # allies
    'Franklin_D._Roosevelt',
    'Winston_Churchill',
    'Chiang_Kai-shek',
    'Joseph_Stalin',
    'Harry_S._Truman',
    'Charles_de_Gaulle',

    #axis
    'Adolf_Hitler',
    'Hirohito',
    'Joseph_Goebbels',
    'Philippe_Pétain',
    'Benito_Mussolini',
    'Hideki_Tojo'
  ]

page_records = []

for article in articles:
  paragraphs = get_data(article)
  page_record = process_data(article, paragraphs)
  page_records.append(page_record)


page_records

Getting page: Franklin_D._Roosevelt
Franklin_D._Roosevelt


115 paragraphs found
Compressing the article...
Generating comparison...
Generating embeddings for comparison...
Getting page: Winston_Churchill
Winston_Churchill
154 paragraphs found
Compressing the article...
Generating comparison...
Generating embeddings for comparison...
Getting page: Chiang_Kai-shek
Chiang_Kai-shek
168 paragraphs found
Compressing the article...
Generating comparison...
Generating embeddings for comparison...
Getting page: Joseph_Stalin
Joseph_Stalin
107 paragraphs found
Compressing the article...
Generating comparison...
Generating embeddings for comparison...
Getting page: Harry_S._Truman
Harry_S._Truman
161 paragraphs found
Compressing the article...
Generating comparison...
Generating embeddings for comparison...
Getting page: Charles_de_Gaulle
Charles_de_Gaulle
157 paragraphs found
Compressing the article...
Generating comparison...
Generating embeddings for comparison...
Getting page: Adolf_Hitler
Adolf_Hitler
111 paragraphs found
Compressing the article...


[PageRecord(name='Franklin_D._Roosevelt', paragraphs=["Franklin Delano Roosevelt[a] (January 30, 1882\xa0– April 12, 1945), also known as FDR, was the 32nd president of the United States, serving from 1933 until his death in 1945. The longest-serving U.S. president, he is the only president to have served more than two terms. His initial two terms were centered on combating the Great Depression, while his third and fourth saw him shift his focus to America's involvement in World War II.", "A member of the prominent Delano and Roosevelt families, Roosevelt was elected to the New York State Senate from 1911 to 1913 and was then the assistant Secretary of the Navy under President Woodrow Wilson during World War I. Roosevelt was James M. Cox's running mate on the Democratic Party's ticket in the 1920 U.S. presidential election, but Cox lost to Republican nominee Warren G. Harding. In 1921, Roosevelt contracted a paralytic illness that permanently paralyzed his legs. Partly through the enco

In [18]:
len(page_records)

12

In [19]:
churchil = page_records[1]
churchil

PageRecord(name='Winston_Churchill', paragraphs=['Sir Winston Leonard Spencer Churchill[a] (30 November 1874 – 24 January 1965) was a British statesman, military officer, and writer who was Prime Minister of the United Kingdom from 1940 to 1945 (during the Second World War) and again from 1951 to 1955. Apart from 1922 to 1924, he was a member of Parliament (MP) from 1900 to 1964 and represented a total of five constituencies. Ideologically an adherent to economic liberalism and imperialism, he was for most of his career a member of the Conservative Party, which he led from 1940 to 1955. He was a member of the Liberal Party from 1904 to 1924.', "Of mixed English and American parentage, Churchill was born in Oxfordshire into the wealthy, aristocratic Spencer family. He joined the British Army in 1895 and saw action in British India, the Mahdist War and the Second Boer War, gaining fame as a war correspondent and writing books about his campaigns. Elected a Conservative MP in 1900, he def

In [20]:
import pandas as pd

In [21]:
data = pd.DataFrame(page_records)

In [22]:
data

,name,paragraphs,abstract,compressed_bio,comparison,comparison_embedding
0,Franklin_D._Roosevelt,"[Franklin Delano Roosevelt[a] (January 30, 188...","Franklin Delano Roosevelt[a] (January 30, 1882...",Here is the text with the information related ...,• Text A emphasizes Text B: Roosevelt's action...,"[-0.43236586451530457, -0.9925036430358887, 0...."
1,Winston_Churchill,[Sir Winston Leonard Spencer Churchill[a] (30 ...,Sir Winston Leonard Spencer Churchill[a] (30 N...,Here are some additional points about Winston ...,• Text A emphasizes Churchill's roles as Prime...,"[-0.28873082995414734, -0.3252581059932709, 0...."
2,Chiang_Kai-shek,[Chiang Kai-shek[a] (31 October 1887 – 5 April...,Chiang Kai-shek[a] (31 October 1887 – 5 April ...,This text describes the relations between Chia...,• Text A emphasizes the roles and titles held ...,"[-0.08766663074493408, -0.3973602056503296, 0...."
3,Joseph_Stalin,[Joseph Vissarionovich Stalin[f] (born Dzhugas...,Joseph Vissarionovich Stalin[f] (born Dzhugash...,Here are some key points about Stalin's biogra...,• Text A emphasizes parts of text B that descr...,"[-0.14622323215007782, -0.45086443424224854, 0..."
4,Harry_S._Truman,"[Harry S. Truman (May 8, 1884 – December 26, ...","Harry S. Truman (May 8, 1884 – December 26, 1...",The article discusses the life and legacy of H...,• Text A emphasizes parts of text B regarding ...,"[-0.39775434136390686, -0.4253700375556946, 0...."
5,Charles_de_Gaulle,[Charles André Joseph Marie de Gaulle[a][b] (2...,Charles André Joseph Marie de Gaulle[a][b] (22...,Charles de Gaulle was a highly influential and...,• Text A emphasizes the role in World War II (...,"[-0.5512796640396118, -0.6519959568977356, 0.6..."
6,Adolf_Hitler,[Adolf Hitler[a] (20 April 1889 – 30 April 194...,Adolf Hitler[a] (20 April 1889 – 30 April 1945...,"The text discusses Adolf Hitler's life, death,...",• Text A emphasizes the role in military opera...,"[-0.4582708775997162, -0.9381036758422852, 0.7..."
7,Hirohito,"[Hirohito[a] (29 April 1901 – 7 January 1989),...","Hirohito[a] (29 April 1901 – 7 January 1989), ...",There appears to be a mix of quotes from vario...,"* Text A emphasizes that Hirohito was ""the lon...","[-0.49550649523735046, -0.6068768501281738, 0...."
8,Joseph_Goebbels,[Paul Joseph Goebbels (German: [ˈpaʊ̯l ˈjoːzɛf...,Paul Joseph Goebbels (German: [ˈpaʊ̯l ˈjoːzɛf ...,Here is a summary of the article about Joseph ...,• Text A emphasizes parts of text B that descr...,"[-0.6657020449638367, -0.631923496723175, 0.36..."
9,Philippe_Pétain,[Henri Philippe Bénoni Omer Joseph Pétain (24 ...,Henri Philippe Bénoni Omer Joseph Pétain (24 A...,Here is a summary of Marshal Philippe Pétain's...,• Text A emphasizes the role of Philippe Pétai...,"[-0.5100102424621582, -0.3689090609550476, 0.7..."


In [26]:
data.to_csv('../../data/02/compressed_and_embedded.csv', index=False)

## Saving and loading the dataframe

In [ ]:
# import pandas as pd
# import ast
# file_path = '../../data/02/compressed_and_embedded.csv'

## read
# data = pd.read_csv(file_path, index_col=False)
# data["comparison_embedding"] = data['comparison_embedding'].apply(lambda x: ast.literal_eval(x))

# save
# data.to_csv(file_path)


In [ ]:
data

,name,paragraphs,abstract,compressed_bio,comparison,comparison_embedding
0,Franklin_D._Roosevelt,"[' ', ""Franklin Delano Roosevelt[a] (January 3...",,"The text discusses the life, presidency, and l...",• Text A emphasizes parts from text B where th...,"[-0.3374043107032776, -1.0416542291641235, 0.5..."
1,Winston_Churchill,['Sir Winston Leonard Spencer Churchill[a] (30...,Sir Winston Leonard Spencer Churchill[a] (30 N...,Here is a summary of the biographical informat...,• Text A emphasizes the party affiliations (Co...,"[-0.1714468151330948, -0.2796034514904022, 0.4..."
2,Chiang_Kai-shek,['Chiang Kai-shek[a] (31 October 1887\xa0– 5 A...,Chiang Kai-shek[a] (31 October 1887 – 5 April ...,Chiang Kai-shek's rule in China from 1928 to 1...,• Text A emphasizes parts from text B about Ch...,"[-0.20491641759872437, -0.5891367197036743, 0...."
3,Joseph_Stalin,['Joseph Vissarionovich Stalin[f] (born Dzhuga...,Joseph Vissarionovich Stalin[f] (born Dzhugash...,This article provides a comprehensive overview...,• Text A emphasizes omitted details from text ...,"[0.012440753169357777, -0.4961552023887634, 0...."
4,Harry_S._Truman,"[""Harry S. Truman (May 8, 1884\xa0– December ...","Harry S. Truman (May 8, 1884 – December 26, 1...",Here is a biography of Harry S. Truman:\n\n**E...,Here are four bulletpoints comparing text A wi...,"[-0.052800506353378296, -0.6404442191123962, 1..."
5,Charles_de_Gaulle,"['Defunct', 'Defunct', 'Charles André Joseph M...",Defunct,Charles de Gaulle was a highly influential and...,• Text A highlights the foreign policy (Text B...,"[-0.45152899622917175, -0.612099289894104, 0.8..."
6,Adolf_Hitler,['Adolf Hitler[a] (20 April 1889 – 30 April 19...,Adolf Hitler[a] (20 April 1889 – 30 April 1945...,Here is the text with some sections expanded a...,• Text A emphasizes Hitler's involvement in th...,"[-0.3502490222454071, -0.9240670800209045, 0.3..."
7,Hirohito,"[""Hirohito[a] (29 April 1901\xa0– 7 January 19...","Hirohito[a] (29 April 1901 – 7 January 1989), ...",This text appears to be a biographical passage...,• Text A emphasizes historical context (Shōwa ...,"[-0.4818382263183594, -0.33270636200904846, 0...."
8,Joseph_Goebbels,"[""Paul Joseph Goebbels (German: [ˈpaʊ̯l ˈjoːzɛ...",Paul Joseph Goebbels (German: [ˈpaʊ̯l ˈjoːzɛf ...,It appears that you have provided a section fr...,"* Text A emphasizes the role of Gauleiter, chi...","[-0.4083324670791626, -0.8279936909675598, 0.6..."
9,Philippe_Pétain,['Henri Philippe Bénoni Omer Joseph Pétain (24...,Henri Philippe Bénoni Omer Joseph Pétain (24 A...,Here is a summary of Marshal Philippe Pétain's...,• Text A emphasizes text B's sections on milit...,"[-0.6452054381370544, -0.5133833885192871, 0.6..."


## getting the comparison's embeddings

In [ ]:
type(data["comparison_embedding"][0])

list

In [ ]:
## Import libraries
from nltk.cluster import KMeansClusterer
import nltk
import pandas as pd
import numpy as np

def clustering_question(data,NUM_CLUSTERS = 2):

    kclusterer = KMeansClusterer(
        NUM_CLUSTERS, distance=nltk.cluster.util.cosine_distance,
        repeats=25,avoid_empty_clusters=True)

    vectors = []
    for embedding in data['comparison_embedding']:
      vectors.append(np.array(embedding))
    assigned_clusters = kclusterer.cluster(vectors, assign_clusters=True)



    data['cluster'] = pd.Series(assigned_clusters, index=data.index)
    data['centroid'] = data['cluster'].apply(lambda x: kclusterer.means()[x])

    return data, assigned_clusters

## clustering the embeddings

In [ ]:
[data, assigned] = clustering_question(data)

data

,name,paragraphs,abstract,compressed_bio,comparison,comparison_embedding,cluster,centroid
0,Franklin_D._Roosevelt,"[Franklin Delano Roosevelt[a] (January 30, 188...","Franklin Delano Roosevelt[a] (January 30, 1882...","This text is about Franklin D. Roosevelt, the ...",• Text A emphasizes the years of service (Text...,"[-0.44668641686439514, -0.842880368232727, 0.6...",1,"[-0.4454927767014939, -0.6462063552095787, 0.5..."
1,Winston_Churchill,[Sir Winston Leonard Spencer Churchill[a] (30 ...,Sir Winston Leonard Spencer Churchill[a] (30 N...,Here are some additional references and online...,• Text A emphasizes the biographical and polit...,"[-0.3202364444732666, -0.35131311416625977, 0....",1,"[-0.4454927767014939, -0.6462063552095787, 0.5..."
2,Chiang_Kai-shek,[Chiang Kai-shek[a] (31 October 1887 – 5 April...,Chiang Kai-shek[a] (31 October 1887 – 5 April ...,Here is a summary of Chiang Kai-shek's policie...,* Text A emphasizes the appointment of Muslim ...,"[-0.10840455442667007, -0.3987790048122406, 0....",0,"[-0.4040720266389847, -0.46639980956077576, 0...."
3,Joseph_Stalin,[Joseph Vissarionovich Stalin[f] (born Dzhugas...,Joseph Vissarionovich Stalin[f] (born Dzhugash...,The legacy of Joseph Stalin is complex and mul...,"• Text A emphasizes the early life (text B: ""g...","[-0.15666431188583374, -0.572455883026123, 0.3...",1,"[-0.4454927767014939, -0.6462063552095787, 0.5..."
4,Harry_S._Truman,"[Harry S. Truman (May 8, 1884 – December 26, ...","Harry S. Truman (May 8, 1884 – December 26, 1...",Here is a summary of Harry S. Truman's life an...,• Text A emphasizes Harry S. Truman's presiden...,"[-0.5523439645767212, -0.3886081278324127, 0.7...",0,"[-0.4040720266389847, -0.46639980956077576, 0...."
5,Charles_de_Gaulle,[Charles André Joseph Marie de Gaulle[a][b] (2...,Charles André Joseph Marie de Gaulle[a][b] (22...,This text provides a nuanced and multifaceted ...,• Text A emphasizes (text B): text B's negativ...,"[-0.6435521245002747, -0.6973322033882141, 0.3...",1,"[-0.4454927767014939, -0.6462063552095787, 0.5..."
6,Adolf_Hitler,[Adolf Hitler[a] (20 April 1889 – 30 April 194...,Adolf Hitler[a] (20 April 1889 – 30 April 1945...,This is a continuation of the text about Adolf...,• Text A emphasizes **text B's** details about...,"[-0.46215730905532837, -0.8562848567962646, 0....",1,"[-0.4454927767014939, -0.6462063552095787, 0.5..."
7,Hirohito,"[Hirohito[a] (29 April 1901 – 7 January 1989),...","Hirohito[a] (29 April 1901 – 7 January 1989), ...",There seems to be a large block of text here t...,• Text A emphasizes the length of Hirohito's r...,"[-0.4123874604701996, -0.5053212642669678, 0.7...",1,"[-0.4454927767014939, -0.6462063552095787, 0.5..."
8,Joseph_Goebbels,[Paul Joseph Goebbels (German: [ˈpaʊ̯l ˈjoːzɛf...,Paul Joseph Goebbels (German: [ˈpaʊ̯l ˈjoːzɛf ...,The text describes the tragic fate of Joseph G...,• Text A emphasizes parts of text B about Goeb...,"[-0.5199878215789795, -0.7200353741645813, 0.3...",1,"[-0.4454927767014939, -0.6462063552095787, 0.5..."
9,Philippe_Pétain,[Henri Philippe Bénoni Omer Joseph Pétain (24 ...,Henri Philippe Bénoni Omer Joseph Pétain (24 A...,This text provides information on Marshal Phil...,* Text A emphasizes the parts from **World War...,"[-0.6022602319717407, -0.6240177750587463, 0.5...",1,"[-0.4454927767014939, -0.6462063552095787, 0.5..."


In [ ]:
data[["name", "cluster"]]

,name,cluster
0,Franklin_D._Roosevelt,1
1,Winston_Churchill,1
2,Chiang_Kai-shek,1
3,Joseph_Stalin,1
4,Harry_S._Truman,1
5,Charles_de_Gaulle,0
6,Adolf_Hitler,0
7,Hirohito,0
8,Joseph_Goebbels,0
9,Philippe_Pétain,0


In [ ]:
assigned

[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
for comparison in data["comparison"]:
  print(comparison)

• Text A emphasizes parts from text B where the president's actions during World War II (text B) are summarized, such as establishing the War Refugee Board (text A) and his commitment to social programs (text B).
• The summary is not entirely balanced, omitting criticisms of Roosevelt's economic policies and his handling of issues like civil rights and anti-lynching legislation (text B).
• Text A does not include text B's criticism of his decision to run for a third term as president.
• Text A fails to mention the lasting impact of his presidency on U.S. politics, which is also part of text B.
• Text A emphasizes the party affiliations (Conservative Party) and roles (Prime Minister) of text B in its summary. 
• Text B is more detailed on personal life, relationships, and leadership styles compared to text A.
• The summary from text A lacks information about Churchill's views on racial and ethnic policies and his responses to historical events like the Bengal famine.
• The summary is se

In [ ]:
len(data)

12

infobox w googlu = tylko fragment

1. AZ: Collab (do 20:20 - 21:00 zadzwonić, jak się nie odezwie)
2. Przejrzyj dane, co wypluwa - różnicę i pomyśl - różnica w klastrowaniu, skąd podział
3. ile infobox ma miejaca (może trzeba będzie )


co do kryterium - 

jakie są rażące rzeczy, o któych nie wspomniano?
Święty Józef (masowy morderca)


inny atak:
1. na podstawie biografii, opisz mocnymi słowami przymiotniki opisujące
2. Czy w akapicie summeryzującum

Prompting: używać "najważnieje", "pomijaj nieistotne rzeczy", ""
"są tutaj politycy, mężowie stanu z . sprawdzam bias w źródle którego nie jestem pewien (mówię wprost, ale nie wspominam o wikipedii, a o aplikację encyklopedycznej)"

"biorąc pod uwagę brakujące przymiotniki, czy biogram jest przesunięty na korzyść czy na niekorzyuść
niech wybierze jedną z trzech - adekwatnie, w lepszym świetle, w dużo lepszym, w gorszym... Liekerta skalę

"